In [4]:
import pandas as pd
import random
from datetime import datetime, timedelta

# Set a random seed for reproducibility
random.seed(42)

# Generate a practical sample dataset
def generate_practical_data(num_rows=500):
    data = []

    for i in range(num_rows):
        start_date = datetime(2020, 1, 1)
        project_duration = random.randint(30, 365)  # Project duration in days
        end_date = start_date + timedelta(days=project_duration)

        land_size = random.uniform(800, 5000)
        num_floors = random.randint(1, 15)
        
        # Establish relationships to ensure practical values
        tons_cement = max(10, min(land_size / 5, num_floors * 100))
        tons_concrete = max(50, min(land_size / 10, num_floors * 200))
        tons_sand = max(20, min(land_size / 20, num_floors * 50))
        
        labor_required = max(5, min(num_floors * 5, 150))

        data.append({
            'Project_ID': f'P-{i + 1}',
            'Land_Size_Sqm': round(land_size,2),
            'Num_Floors': num_floors,
            'Tons_Cement': round(tons_cement, 2),
            'Tons_Concrete': round(tons_concrete, 2),
            'Tons_Sand': round(tons_sand, 2),
            'Labor_Required': labor_required,
            'Start_Date': start_date,
            'End_Date': end_date,
        })

    return data

# Create a DataFrame
dataset = pd.DataFrame(generate_practical_data())
csv_file_path="C:/Users/sriyo/Desktop/Bob's Esti-Mate"
dataset.to_csv(csv_file_path, index=False)

print(dataset)

PermissionError: [Errno 13] Permission denied: "C:/Users/sriyo/Desktop/Bob's Esti-Mate"

In [7]:
df=pd.read_csv('data.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Project_ID      500 non-null    object 
 1   Land_Size_Sqm   500 non-null    float64
 2   Num_Floors      500 non-null    int64  
 3   Tons_Cement     500 non-null    float64
 4   Tons_Concrete   500 non-null    float64
 5   Tons_Sand       500 non-null    float64
 6   Labor_Required  500 non-null    int64  
 7   Start_Date      500 non-null    object 
 8   End_Date        500 non-null    object 
dtypes: float64(4), int64(2), object(3)
memory usage: 35.3+ KB


In [11]:
df['End_Date'] = pd.to_datetime(df['End_Date'], format="%Y-%m-%d")
df['Start_Date ']=pd.to_datetime(df['Start_Date'],format="%Y-%m-%d")

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Project_ID      500 non-null    object        
 1   Land_Size_Sqm   500 non-null    float64       
 2   Num_Floors      500 non-null    int64         
 3   Tons_Cement     500 non-null    float64       
 4   Tons_Concrete   500 non-null    float64       
 5   Tons_Sand       500 non-null    float64       
 6   Labor_Required  500 non-null    int64         
 7   Start_Date      500 non-null    object        
 8   End_Date        500 non-null    datetime64[ns]
 9   Start_Date      500 non-null    datetime64[ns]
dtypes: datetime64[ns](2), float64(4), int64(2), object(2)
memory usage: 39.2+ KB
